# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **FPL player score** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Batch Inference</span>

## 🗒️ This notebook is divided into the following sections:

1. Download model and batch inference data
2. Make predictions, combine predictions with static data
3. Store predictions in a monitoring feature group

## <span style='color:#ff5f27'> 📝 Imports

In [49]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from functions import util
import os
from importlib import reload
import requests

In [ ]:
reload(util)

print(util.get_gameweek_from_date("2025-1-4"))

## <span style="color:#ff5f27;"> 📡 Connect to Hopsworks Feature Store </span>

In [ ]:
try:
    with open('./hopsworks/hopsworks-api-key.txt', 'r') as file:
        os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
except:
    print("In production mode")

project = hopsworks.login()
fs = project.get_feature_store()

## <span style="color:#ff5f27;">🪝 Download the model from Model Registry</span>

In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="player_score_xgboost_model",
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

In [54]:
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
general_info = requests.get(bootstrap_url).json()

In [ ]:
try:
    monitor_fg = fs.get_feature_group(
        name='fpl_predictions',
    )
    prev_gameweek_nr = int(monitor_fg.select(["gameweek"]).read().max())
except:
    print("Feature group not found, use player features instead")
    player_fg = fs.get_feature_group("player_features")
    prev_gameweek_nr = int(player_fg.select(["gameweek"]).read().max()) - 1

In [ ]:
prev_gameweek_nr

In [ ]:
prev_gameweek = next(
    (gw for gw in general_info['events'] if gw['id'] == prev_gameweek_nr), None
)

prev_finished = prev_gameweek['finished']
prev_finished

## <span style="color:#ff5f27;">✨ Get Player Score Features From Previous Gameweek </span>



In [ ]:
if prev_finished:
    players_fg = fs.get_feature_group(
        name='player_features',
        version=1,
    )
    players = players_fg.filter(players_fg.gameweek == prev_gameweek_nr + 1).read()
    print("get stats for finished gameweek")

In [ ]:
if prev_finished:

    try:
        monitor_fg = fs.get_feature_group(
            name='fpl_predictions',
        )

        existing_data = monitor_fg.read()

        existing_data = existing_data.set_index('id')
        players = players.set_index('id')

        # Update the "points" column for the specified gameweek
        existing_data.loc[existing_data['gameweek'] == prev_gameweek_nr + 1, 'points'] = players['points']

        # Reset index if needed
        existing_data = existing_data.reset_index()
        players = players.reset_index()
        
        # Save the updated data back to the feature group
        monitor_fg.insert(existing_data[existing_data["gameweek"] == prev_gameweek_nr + 1])
        print(f"Updated player actual points for gameweek {prev_gameweek_nr + 1}")
    except:
        print("No previous predictions")


### <span style="color:#ff5f27;">🤖 Making the predictions</span>

In [ ]:
inputs = ['prev_minutes', 'prev_goals_scored', 'prev_assists',
         'prev_clean_sheets', 'prev_goals_conceded', 'prev_own_goals',
         'prev_penalties_saved', 'prev_penalties_missed', 'prev_yellow_cards',
         'prev_red_cards', 'prev_saves', 'prev_bonus',
         'prev_bps', 'prev_influence', 'prev_creativity',
         'prev_threat', 'prev_ict_index', 'prev_starts',
         'prev_expected_goals', 'prev_expected_assists', 'prev_expected_goal_involvements',
         'prev_expected_goals_conceded', 'prev_in_dreamteam', 'prev_total_points',
         'prev_selected', 'prev_transfers_balance', 'prev_value', 'prev_was_home', 
         'prev_opponent_team','element_type']

#for i in range(0, len(input_data)):
#    output_data.at[i, 'predicted_score'] = retrieved_xgboost_model.predict(input_data.loc[i, inputs].values.reshape(1, -1))[0]

if prev_finished:
    players['predicted_score'] = retrieved_xgboost_model.predict(players[inputs]).astype("float64")
    print("made predictions")

### <span style="color:#ff5f27;">🤖 Saving the predictions (for monitoring) to a Feature Group</span>

### Create Score Graph
Draw a graph of the predictions with all the players as a PNG and save it to the github

In [ ]:
if prev_finished:
    # Get or create feature group
    monitor_fg = fs.get_or_create_feature_group(
        name='fpl_predictions',
        description='FPL player score prediction monitoring',
        version=1,
        primary_key=['id', 'gameweek']
    )
    print("Got the monitor fg")

In [ ]:
if prev_finished:
    print(f"Predicted for gameweek {prev_gameweek_nr + 1}")
    # Only keep the relevant columns
    output_data = players[['id', 'points', 'gameweek', 'predicted_score', 'prev_total_points', 'prev_value']]

In [ ]:
if prev_finished:
    # Add static data
    for i in range(0, len(output_data)):
        player_id = output_data.loc[i, 'id']

        player_data = util.get_player_info(player_id, general_info)
        for key in player_data:
            output_data.loc[i, key] = player_data[key]
    print("Added static data")

In [64]:
if prev_finished and len(output_data) > 0:
    monitor_fg.insert(output_data)
    print("Inserted the data")

In [ ]:
if prev_finished:
    monitor_fg.update_feature_description("id", "Player ID")
    monitor_fg.update_feature_description("points", "Total points of the player in the gameweek (label)")
    monitor_fg.update_feature_description("gameweek", "Gameweek")
    monitor_fg.update_feature_description("predicted_score", "Predicted score of the player in the gameweek")
    monitor_fg.update_feature_description("first_name", "Player first name")
    monitor_fg.update_feature_description("second_name", "Player surname")
    monitor_fg.update_feature_description("team", "Player team")
    monitor_fg.update_feature_description("position", "Player position")
    monitor_fg.update_feature_description("prev_total_points", "Total points of the player in the previous gameweek")
    monitor_fg.update_feature_description("prev_value", "Value of the player in the previous gameweek")
    print("Updated the descriptions")

---